In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np

# 데이터셋 경로 설정
dataset_path = "F:/images"

# ImageDataGenerator를 사용하여 이미지 전처리 및 데이터 증강 설정
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 검증 데이터셋 분할 비율

# ImageDataGenerator를 사용하여 폴더에서 이미지를 로드하고 데이터셋 생성
image_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    classes=["elliptical", "spiral", "uncertain"],
    subset='training')  # 훈련 데이터셋 설정

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    classes=["elliptical", "spiral", "uncertain"],
    subset='validation')  # 검증 데이터셋 설정

# 모델 정의
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    #Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')
])

# Display the model's architecture
model.summary()

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
EPOCHS = 10
history = model.fit(image_generator, epochs=EPOCHS, validation_data=validation_generator)

#module save
model.save("C:/Users/user/python_machine_learning/sample_model.h5")

#reconstructed_model = keras.models.load_model("my_model")

# assess model
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


num_images_to_show = 10
random_indexes = np.random.randint(0, len(validation_generator), num_images_to_show)



class_names = ["elliptical", "spiral", "uncertain"]

for i, index in enumerate(random_indexes, 1):
    plt.subplot(2, 5, i)
    images, labels = validation_generator[index]
    test_image = images[0] * 255  # 이미지를 정규화되지 않은 상태로 출력
    true_label = class_names[np.argmax(labels[0])]  # 실제 클래스 레이블
    input_image = np.expand_dims(test_image, axis=0)
    predicted_scores = model.predict(input_image)
    predicted_label = class_names[np.argmax(predicted_scores)]  # 모델의 예측 결과

    # 이미지 및 예측 결과 시각화
    plt.imshow(test_image.astype('uint8'))  # 이미지를 정수로 변환하여 출력
    plt.title(f'True: {true_label}\nPredicted: {predicted_label}', size = 8)
    plt.axis('off')

plt.show()

Found 34585 images belonging to 3 classes.
Found 8645 images belonging to 3 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 256)       295168    
                                   

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions, MobileNetV2
import numpy as np

# 모델 로드
model = tf.keras.models.load_model("C:/Users/user/python_machine_learning/sample_model.h5", compile=False)
model = MobileNetV2(input_shape=(224,224,3), include_top=True)

# 예측할 이미지 경로
image_path = "C:/Users/user/python_machine_learning/2023_창의학술제/sample_image.jpg"

# 이미지 로드 및 전처리
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# 모델을 사용하여 예측
predictions = model.predict(img_array)

# 예측 결과 출력
decoded_predictions = decode_predictions(predictions, top=3)[0]
decoded_predictions
# for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
#     print(f"{i + 1}: {label} ({score:.2f})")

# 예측된 클래스 또는 확률 등을 사용하여 원하는 작업을 수행
# 예를 들어, 가장 높은 확률을 가진 클래스 출력
# top_prediction = decoded_predictions[0][1]
# print(f"\nTop Prediction: {top_prediction}")

1/1 [==============================] - 1s 685ms/step


[('n03982430', 'pool_table', 0.2090479),
 ('n03942813', 'ping-pong_ball', 0.06912012),
 ('n04286575', 'spotlight', 0.02605729)]